<a href="https://colab.research.google.com/github/ugursirvermez/PyTorch_Education/blob/main/08_pytorch_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Transfer Learning

##Aşama-1: Kurulum ve Güncelleme

In [2]:
# Torch'un güncel olarak torch 1.12+ ve torchvision 0.13+ versiyonlarından üstün olup olamdığına bakalım.
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version 1.12+ üstü olmalıdır!"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version 0.13+ üstü olmalıdır!"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versionları son sürümleri ile güncellenmektedir...")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versionları son sürümleri ile güncellenmektedir...
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
torch version: 2.5.0+cu121
torchvision version: 0.20.0+cu121


##Kütüphaneleri Getirme

In [3]:
# Temel Kütüphaneler
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Torchinfo var mı? yoksa Yükle!
try:
    from torchinfo import summary
except:
    print("[INFO] Torchinfo bulunamadı. Yükleniyor...")
    !pip install -q torchinfo
    from torchinfo import summary

# Module_Files dosyasındaki kütüphanelerimizi yüklemeye çalışalım.
try:
    from Module_Files import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Dosyalar bulunamadı. Github'tan dosyalara erişiliyor...")
    !git clone https://github.com/ugursirvermez/PyTorch_Education/
    !mv PyTorch_Education/Module_Files .
    !rm -rf PyTorch_Education
    from Module_Files import data_setup, engine

[INFO] Torchinfo bulunamadı. Yükleniyor...
[INFO] Dosyalar bulunamadı. Github'tan dosyalara erişiliyor...
Cloning into 'PyTorch_Education'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 133 (delta 65), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (133/133), 8.93 MiB | 14.14 MiB/s, done.
Resolving deltas: 100% (65/65), done.
data/pizza_steak_sushi Dosyası olusturuluyor
İndiriliyor...
Sıkıştırılmış dosyada çıkarılıyor...
Train data:
Dataset ImageFolder
    Number of datapoints: 225
    Root location: data/pizza_steak_sushi/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 75
    Root location: data/pizza_steak_sushi/test
    StandardTransform
Transform: Compose(
              

### device ? cpu : gpu -> cihaza yolla

In [4]:
# device ? cpu : gpu -> cihaza yolla
device = "cuda" if torch.cuda.is_available() else "cpu"

##Aşama-2: Veri Alma Aşaması

In [5]:
import os
import zipfile

from pathlib import Path

import requests

# Dosya yolunu oluştur.
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# Yoksa bir yol hazırla!
if image_path.is_dir():
    print(f"{image_path} bu yol zaten var.")
else:
    print(f"{image_path} dosya yolu bulunamadı. Bir tane oluştur...")
    image_path.mkdir(parents=True, exist_ok=True)

    # David Bourke'un oluşturduğu veri yolunu oluştur.
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Veriler indiriliyor...")
        f.write(request.content)

    # Veriler arşivden ayıklanıyor...
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Veriler arşivden ayıklanıyor...")
        zip_ref.extractall(image_path)

    # Remove .zip file
    os.remove(data_path / "pizza_steak_sushi.zip")

data/pizza_steak_sushi bu yol zaten var.


###Dosya Yolu Oluştur.

In [6]:
#Dosya yollarını sadeleştirmek için değişkene aktarma yap.
train_dir = image_path / "train"
test_dir = image_path / "test"

##DataSet ve DataLoader Oluştur.

In [7]:
# Önceden eğitilmiş veri için uygun veri sağla.
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = ImageNet için önceden eğitilmiş verilere uygun en ideal değerdir.
# Bu veri için otomatik en uygun transform oluştur.
auto_transforms = weights.transforms()

# DataLoader'ları oluşturma ve Etiketleri isimlendirme
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=auto_transforms, #Otomatik doldurma yapılacak
                                                                               batch_size=32) # mini-batch miktarını 32 yap.

##Aşama-3: Eğitilmiş Modeli Alma